# Assignment #6: Dependency parsing
Author: Pierre Nugues

## Objectives

This assignment is inspired by the CoNLL 2018 shared task of the conference on computational natural language learning on dependency parsing, http://universaldependencies.org/conll18/. It is a follower of <a href="http://ilk.uvt.nl/conll/">CONLL-X</a>, which was the first large-scale evaluation of dependency parsers.
            
In this session, you will implement a dependency parser for Swedish and, optionally, another language that you will choose.

The objectives of this assignment are to:
* Know what a dependency graph is
* Understand the principles of a transition-based parser
* Extend the parser with a guiding predicate that parses an annotated dependency graph
* Extract features to learn parsing actions from an annotated corpus
* Write a short report on your results

## Organization and location

You can work alone or collaborate with another student.
Each group will have to:
* Write a program that parses a sentence when the dependency graph is known
* Extract features from the parsing actions.
* Train a classifier
* Apply it on a test corpus
* Evaluate the results

The column names of the CoNLL-U corpora

In [3]:
column_names_u = ['ID', 'FORM', 'LEMMA', 'UPOS', 'XPOS', 'FEATS', 'HEAD', 'DEPREL', 'DEPS', 'MISC']

#### Functions to read the CoNLL-U files

In [4]:
def read_sentences(file):
    """
    Creates a list of sentences from the corpus
    Each sentence is a string
    :param file:
    :return:
    """
    f = open(file, encoding='utf-8').read().strip()
    sentences = f.split('\n\n')
    return sentences

In [5]:
def split_rows(sentences, column_names):
    """
    Creates a list of sentence where each sentence is a list of lines
    Each line is a dictionary of columns
    :param sentences:
    :param column_names:
    :return:
    """
    new_sentences = []
    root_values = ['0', 'root', 'root', 'root', 'root', 'root', '0', 'root', 'root', 'root']
    start = [dict(zip(column_names, root_values))]
    for sentence in sentences:
        rows = sentence.split('\n')
        sentence = [dict(zip(column_names, row.split('\t'))) for row in rows if row[0] != '#']
        sentence = start + sentence
        new_sentences.append(sentence)
    return new_sentences

#### Reading the corpus

We load the Swedish _Talbanken_ corpus.

In [6]:
sentences = read_sentences(path_sv_train)
formatted_corpus_train = split_rows(sentences, column_names_u)

The parsed sentence: _Individuell beskattning av arbetsinkomster_

#### Removing indices that are not integers

To ease the processing of some corpora, we remove the indices which are not integers. We do this because `ID` is not necessarily a number.

In [9]:
def clean_indicies(formatted_corpus):
    formatted_corpus_clean = []
    for sentence in formatted_corpus:
        formatted_corpus_clean.append([word for word in sentence if word['ID'].isdigit()])
    return formatted_corpus_clean          

## Transition parser

For each sentence with a projective dependency graph, there is an action sequence that enables the transition parser
to generate this graph. Gold standard parsing corresponds to the sequence of parsing actions, left-arc (<tt>la</tt>), right-arc (<tt>ra</tt>), shift (<tt>sh</tt>), and reduce (<tt>re</tt>) that produces the manually-obtained, gold standard, graph.

### The transitions

Here are implementations of the parsing transitions. Read them and be sure you understand them.

In [11]:
def shift(stack, queue, graph):
    """
    Shift the first word in the queue onto the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    stack = [queue[0]] + stack
    queue = queue[1:]
    return stack, queue, graph


def reduce(stack, queue, graph):
    """
    Remove the first item from the stack
    :param stack:
    :param queue:
    :param graph:
    :return:
    """
    return stack[1:], queue, graph


def right_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the top of the stack to the first in the queue
    and shifts
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][queue[0]['ID']] = stack[0]['ID']
    if deprel:
        graph['deprels'][queue[0]['ID']] = deprel
    else:
        graph['deprels'][queue[0]['ID']] = queue[0]['DEPREL']
    # If we create an arc from the 'root', we introduce a statement to pop it to avoid multiple roots
    if stack[0]['ID'] == '0':
        stack = stack[1:]
    return shift(stack, queue, graph)


def left_arc(stack, queue, graph, deprel=False):
    """
    Creates an arc from the first in the queue to the top of the stack
    and reduces it.
    The deprel argument is either read from the manually-annotated corpus
    (deprel=False) or assigned by the parser. In this case, the deprel
    argument has a value
    :param stack:
    :param queue:
    :param graph:
    :param deprel: either read from the manually-annotated corpus (value false)
    or assigned by the parser
    :return:
    """
    graph['heads'][stack[0]['ID']] = queue[0]['ID']
    if deprel:
        graph['deprels'][stack[0]['ID']] = deprel
    else:
        graph['deprels'][stack[0]['ID']] = stack[0]['DEPREL']        
    return reduce(stack, queue, graph)

### Constrains on the transitions

We add a few constraints before we carry out the transitions. Given a manually-annotated dependency graph, look at the conditions (`can_...()` functions) on the stack and the current input list -- the queue -- to execute left-arc, right-arc, shift, or reduce.

In [12]:
def can_reduce(stack, graph):
    """
    Checks that the top of the stack has a head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return True
    else:
        return False

    
def can_leftarc(stack, graph):
    """
    Checks that the top of the has no head
    :param stack:
    :param graph:
    :return:
    """
    if not stack:
        return False
    if stack[0]['ID'] in graph['heads']:
        return False
    else:
        return True


def can_rightarc(stack):
    """
    Simply checks there is a stack
    :param stack:
    :return:
    """
    if not stack:
        return False
    else:
        return True

### Finding the transitions from a manually-parsed sentence

Using an annotated corpus, we can derive the action sequences producing the manually-parsed sentences (provided that they are projective). We use an oracle for this as explained during the lectures.

In [13]:
def oracle(stack, queue, graph):
    """
    Gold standard parsing
    Produces a sequence of transitions from a manually-annotated corpus:
    sh, re, ra.deprel, la.deprel
    :param stack: The stack
    :param queue: The input list
    :param graph: The set of relations already parsed
    :return: the transition and the grammatical function (deprel) in the
    form of transition.deprel
    """
    # Right arc
    if stack and stack[0]['ID'] == queue[0]['HEAD']:
        # print('ra', queue[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + queue[0]['DEPREL']
        stack, queue, graph = right_arc(stack, queue, graph)
        return stack, queue, graph, 'ra' + deprel
    # Left arc
    if stack and queue[0]['ID'] == stack[0]['HEAD']:
        # print('la', stack[0]['DEPREL'], stack[0]['UPOS'], queue[0]['UPOS'])
        deprel = '.' + stack[0]['DEPREL']
        stack, queue, graph = left_arc(stack, queue, graph)
        return stack, queue, graph, 'la' + deprel
    # Reduce
    if stack and can_reduce(stack, graph):
        for word in stack:
            if (word['ID'] == queue[0]['HEAD'] or
                    word['HEAD'] == queue[0]['ID']):
                # print('re', stack[0]['UPOS'], queue[0]['UPOS'])
                stack, queue, graph = reduce(stack, queue, graph)
                return stack, queue, graph, 're'
    # Shift
    # print('sh', [], queue[0]['UPOS'])
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

### Dealing with nonprojective graphs

Oracle parsing produces a sequence of transitions if the graph is projective and well-formed. If not, we will have headless words in the stack. Parsing normally terminates when the queue is empty. We also empty the stack to be sure that all the words have a head. We attach headless words to the root word of the sentence.

In [14]:
def exists_root(graph):
    for (x, y)  in graph['heads'].items():
        if y == '0' and x != '0':
            return x
    return False

In [15]:
def empty_stack(stack, graph):
    """
    Pops the items in the stack. If they have no head, they are assigned
    a ROOT head
    :param stack:
    :param graph:
    :return:
    """
    idx_root = exists_root(graph)
    # There is already a root
    if idx_root:
        for word in stack:
            if word['ID'] not in graph['heads']:
                graph['heads'][word['ID']] = idx_root
                graph['deprels'][word['ID']] = 'dep'
    else:
        # There is no root. We assign the root to the first headless word.
        for word in stack:
            if word['ID'] not in graph['heads']:
                if idx_root:
                    graph['heads'][word['ID']] = idx_root
                    graph['deprels'][word['ID']] = 'dep'
                else:
                    graph['heads'][word['ID']] = '0'
                    graph['deprels'][word['ID']] = 'root'
                    idx_root = word['ID']
    stack = []
    return stack, graph

### Checking if two graphs are equal

The `equal_graphs()` utility checks if the graph obtained from a sequence of transitions is equal to the annotated graph. It is normally the case, except with nonprojective graphs.

In [24]:
def equal_graphs(sentence, graph, verbose=False):
    """
    Checks that the graph corresponds to the gold standard annotation of a sentence
    :param sentence:
    :param graph:
    :return:
    """
    equal = True
    for word in sentence:
        if word['ID'] in graph['heads'] and word['HEAD'] == graph['heads'][word['ID']]:
            pass
        else:
            equal = False
            if verbose:
                print(word, flush=True)
    return equal

### Parsing an annotated corpus with an oracle

You will now run the code below. With it, you will produce a sequence of transitions for each sentence. If the graph is projective, applying the sequence to the sentence will recreate the gold-standard annotation.

For this experiment:
1. Understand from the slides used during the lecture how the oracle carries out a gold standard parsing. 
2. The parser can only deal with projective sentences. In the case of a nonprojective one, the parsed graph and the manually-annotated sentence are not equal. Examine one nonprojective sentence (just set `verbose`to `True` in the code below) and explain why it is not projective. Take a short one (the shortest). You will **describe** this in the report.

In [25]:
def init_config(sentence):
    stack = []
    queue = list(sentence)
    graph = {}
    graph['heads'] = {}
    graph['heads']['0'] = '0'
    graph['deprels'] = {}
    graph['deprels']['0'] = 'ROOT'
    return stack, queue, graph

In [ ]:
verbose = True
projectivization = False

transition_corpus = []
graph_corpus = []

for sent_cnt, sentence in enumerate(formatted_corpus_train_clean):
    #print(sentence)
    stack, queue, graph = init_config(sentence)
    transition_sent = []
    while queue:
        stack, queue, graph, trans = oracle(stack, queue, graph)
        transition_sent.append(trans)
    stack, graph = empty_stack(stack, graph)
    transition_corpus.append(transition_sent)
    graph_corpus.append(graph)

    if verbose:
        if not equal_graphs(sentence, graph):
            print('Annotation and gold-standard parsing not equal')
            print('Sentence:', sentence)
            print('Gold-standard graph', graph)
    # Poorman's projectivization to have well-formed graphs.
    # We just just assign the same heads as what gold standard parsing did
    # This guarantee a projective sentence
    if projectivization:
        for word in sentence:
            word['HEAD'] = graph['heads'][word['ID']]
print('\nProcessed ' + str(sent_cnt) + ' sentences')

## Training a classifier

We can now train a classifier to predict an action from a current parsing context. To be able to predict the next action from a given parsing state, gold standard parsing must also extract feature vectors at each step of the parsing procedure. The simplest parsing context corresponds to words' part of speech on the top of the stack and head of the input list (the queue).
    
Once the data collected, the training procedure will produce a 4-class classifier that you will embed in
Nivre's parser to choose the next action. During parsing, Nivre's parser will call the classifier to choose
the next action in the set {<tt>la</tt>, <tt>ra</tt>, <tt>sh</tt>, <tt>re</tt>} using the current context.

You will use two feature sets to build your models:
* The top of the stack and the first word of the input list (word forms and parts of speech);
* The two first words and POS on the top of the stack and the two first words and POS of the input list;
* You will also add constraints to actions. You will encode these constraints as Boolean features.

### Parsing the grammatical functions

Using the actions in the set {<tt>la</tt>, <tt>ra</tt>, <tt>sh</tt>, <tt>re</tt>} produces an unlabelled
graph. It is easy to extend the parser so that it can label the graph with grammatical functions. In this
case, we must complement the actions <tt>la</tt>
and <tt>ra</tt> with their function using this notation for example:<tt>la.mod</tt>, <tt>la.case</tt>, <tt>ra.nmod</tt>, etc. where the prefix is the action and the suffix is the function.

### Extracting features 

The final goal is to parse the Swedish corpus and produce a labelled dependency graph. 

You will consider two feature sets and you will train the corresponding logistic regression models using scikit-learn:
1. The first set will use the word and the part of speech extracted from the first element in the stack and the first in the queue,
2. the second one will use two elements from the stack and two from the input list.

These sets will include two additional Boolean parameters, "can do left arc" and "can do reduce", which will model constraints on the parser's actions. In total, the feature sets will then have six, respectively ten parameters.

This means that the purpose of this assignment is to generate two scikit-learn models for the labelled graphs. We use the depth parameter for this: The depth of the stack and the queue, either 1 or 2. Start with 1.

In [60]:
depth = 2

You will need the `queue_stack()` function.

In [61]:
def queue_stack(queue_or_stack, graph, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    features_deprel = ['nil'] * depth
    if queue_or_stack:
        for i, word in list(enumerate(queue_or_stack))[:depth]:
            features_pos[i] = queue_or_stack[i]['UPOS']
            features_lex[i] = queue_or_stack[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

Optionally, you may want to extend the feature vector with words to the left of the top of the stack with the `right_context()` function. If the top of the stack has index $i$, you will extract the words and their parts of speech at index $i + 1$, $i+2$. This will noticeably improve the performance.

In [62]:
def right_context(stack, sentence, depth, pos=True, lex=True):
    features = []
    features_pos = ['nil'] * depth
    features_lex = ['nil'] * depth
    if stack:
        fw_id = int(stack[0]['ID']) + 1
        for i, word in list(enumerate(sentence))[fw_id: fw_id + depth]:
            features_pos[i - fw_id] = sentence[i]['UPOS']
            features_lex[i - fw_id] = sentence[i]['FORM']
    if pos:
        features += features_pos
    if lex:
        features += features_lex
    return features

The next function returns the features in a dictionary format compatible with scikit-learn. You have a code example of feature encoding in this format in the chunking program.

In [63]:
def extract(depth, stack, queue, graph, sentence):
    """
    :param stack:
    :param queue:
    :param graph:
    :param feature_names:
    :param sentence:
    :return:
    """
    improved = True
    if improved:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             right_context(stack, sentence, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph)])
    else:
        x = (queue_stack(stack, graph, depth) +
             queue_stack(queue, graph, depth) +
             [can_reduce(stack, graph), can_leftarc(stack, graph)])     
    feature_names = ['feat' + str(i) for i in range(len(x))]
    features = dict(zip(feature_names, x))
    return features

Now write a loop to parse the annotated corpus using the oracle and collect the features in a matrix ($\mathbf{X}$) and the transitions in a vector ($\mathbf{y}$). 

The first lines of your features for the 4 parameters ($\mathbf{x}$) and labelled actions ($y$) should look like the excerpt below, where the columns correspond to stack0_POS, stack1_POS, stack0_word, stack1_word, queue0_POS, queue1_POS, queue0_word, queue1_word, can-re, can-la, and the transition value (`depth = 2`):
$\mathbf{X} =
\begin{bmatrix}
\text{nil}& \text{nil} &\text{nil} & \text{nil} & \text{ROOT} & \text{ADJ} & \text{ROOT} & \text{Individuell} & \text{False} & \text{False}\\
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{ADJ} &     \text{NOUN} &     \text{Individuell} &     \text{beskattning} &     \text{True} &     \text{False}\\ 
\text{ADJ} &     \text{ROOT} &     \text{Individuell} &     \text{ROOT} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{False} &     \text{True}\\ 
\text{ROOT} &     \text{nil} &     \text{ROOT} &     \text{nil} &     \text{NOUN} &     \text{ADP} &     \text{beskattning} &     \text{av} &     \text{True} &     \text{False}\\
\text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{ADP} &     \text{NOUN} &     \text{av} &     \text{arbetsinkomster} &     \text{True} &     \text{False}\\
\text{ADP} &     \text{NOUN} &     \text{av} &     \text{beskattning} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{False} &     \text{True}\\  \text{NOUN} &     \text{ROOT} &     \text{beskattning} &     \text{ROOT} &     \text{NOUN} &     \text{nil} &     \text{arbetsinkomster} &     \text{nil} &     \text{True} &  \text{False}
\end{bmatrix}$
; $\mathbf{y} =
\begin{bmatrix}
\text{sh}\\
\text{sh}\\
\text{la.amod}\\
\text{ra.root}\\
\text{sh}\\
\text{la.case}\\
\text{ra.nmod}
\end{bmatrix}$

You will store your matrix in a Python dictionary and the classes in a list

In [64]:
X_dict = []
y_symbols = []

In [65]:
from tqdm import tqdm

def apply_transition(stack, queue, graph, trans):
    if stack and trans[:2] == 'ra':
        stack, queue, graph = right_arc(stack, queue, graph, trans[3:])
        return stack, queue, graph, 'ra'
    if stack and can_leftarc(stack, graph) and trans[:2] == 'la':
        if trans[3:] == 'root' and exists_root(graph):
            stack, queue, graph = shift(stack, queue, graph)
            return stack, queue, graph, 'sh'
        else:
            stack, queue, graph = left_arc(stack, queue, graph, trans[3:])
            return stack, queue, graph, 'la'
    if stack and can_reduce(stack, graph) and trans == 're':
        stack, queue, graph = reduce(stack, queue, graph)
        return stack, queue, graph, 're'
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

for (sentence, transition) in tqdm(zip(formatted_corpus_train_clean, transition_corpus)):
    stack, queue, graph = init_config(sentence)
    i = 0
    while queue:
        X_dict.append(extract(depth, stack, queue, graph, sentence))
        y_symbols.append(transition[i])
        stack, queue, graph, trans = apply_transition(stack, queue, graph, transition[i])
        i += 1

4303it [00:03, 1187.28it/s]


In [ ]:
X_dict[:7]

In [67]:
y_symbols[:7]

['sh', 'sh', 'la.amod', 'ra.root', 'sh', 'la.case', 'ra.nmod']

### Fitting the models

Vectorize your `X_dict` into an `X` matrix using `DictVectorizer()`

In [68]:
from sklearn.feature_extraction import DictVectorizer
from sklearn import linear_model
vec = DictVectorizer()
X = vec.fit_transform(X_dict)

Fit the model. With sklearn, you can use `y_symbols` directly. Use `verbose=True` and `n_jobs=8` or more.

In [69]:
classifier = linear_model.LogisticRegression(verbose=True, n_jobs=8)
model = classifier.fit(X, y_symbols)
model

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   1 out of   1 | elapsed:  2.5min finished


LogisticRegression(n_jobs=8, verbose=True)

## Prediction

Now use this model to predict the sentences in the test corpus

In [ ]:
sentences_test = read_sentences(path_sv_test)
formatted_corpus_test = split_rows(sentences_test, column_names_u)
formatted_corpus_test_clean = clean_indicies(formatted_corpus_test)
formatted_corpus_test_clean[0]

In [71]:
def apply_transition(stack, queue, graph, trans):
    if stack and trans[:2] == 'ra':
        stack, queue, graph = right_arc(stack, queue, graph, trans[3:])
        return stack, queue, graph, 'ra'
    if stack and can_leftarc(stack, graph) and trans[:2] == 'la':
        if trans[3:] == 'root' and exists_root(graph):
            stack, queue, graph = shift(stack, queue, graph)
            return stack, queue, graph, 'sh'
        else:
            stack, queue, graph = left_arc(stack, queue, graph, trans[3:])
            return stack, queue, graph, 'la'
    if stack and can_reduce(stack, graph) and trans == 're':
        stack, queue, graph = reduce(stack, queue, graph)
        return stack, queue, graph, 're'
    stack, queue, graph = shift(stack, queue, graph)
    return stack, queue, graph, 'sh'

In [72]:
from tqdm import tqdm

for sent_cnt, sentence in tqdm(enumerate(formatted_corpus_test_clean)):
    X_test_dict = []
    stack, queue, graph = init_config(sentence)
    while queue:
        X_test_dict = extract(depth, stack, queue, graph, sentence)
        X_test = vec.transform(X_test_dict)
        y_test = classifier.predict(X_test)[0]
        stack, queue, graph, trans = apply_transition(stack, queue, graph, y_test)
    stack, graph = empty_stack(stack, graph)
    for word in sentence:
        word['HEAD'] = graph['heads'][word['ID']]
        word['DEPREL'] = graph['deprels'][word['ID']]

1219it [15:27,  1.31it/s]
